In [132]:
import os
import pandas as pd

# Define the source and destination folders
src_folder = 'src'
csv_folder = 'csv'

# Create the destination folder if it doesn't exist
if not os.path.exists(csv_folder):
    os.makedirs(csv_folder)

# Loop through all files in the source folder
for filename in os.listdir(src_folder):
    # Check if the file is an .xlsx file
    if filename.endswith('.xlsx'):
        # Construct the full file path
        file_path = os.path.join(src_folder, filename)
        
        # Load the Excel file
        excel_data = pd.read_excel(file_path)
        
        # Construct the path for the CSV file
        csv_filename = os.path.splitext(filename)[0] + '.csv'
        csv_path = os.path.join(csv_folder, csv_filename)
        
        # Save the data to a CSV file
        excel_data.to_csv(csv_path, index=False)

print("Conversion complete!")

Conversion complete!


Мы получили CSV, теперь нормализуем. Для начала добавим поле со счетами

In [133]:
import os
import pandas as pd

# Define the destination folder
csv_folder = 'csv'

# List of section headers
section_headers = [
    "101.34",
    "101.36",
    "101.38", 
    "105.34",
    "105.35",
    "105.36",
    "21.34",
    "21.36",
    "21.38",
    "21.34, Машины и оборудование - иное движимое имущество",
    "21.36, Инвентарь производственный и хозяйственный – иное движимое имущество",
    "21.38, Прочие основные средства - иное движимое имущество",
    "101.34, Машины и оборудование – иное движимое имущество учреждения",
    "101.36, Инвентарь производственный и хозяйственный – иное движимое имущество учреждения",
    "101.38, Прочие основные средства – иное движимое имущество учреждения",

]

def process_csv_file(file_path):
    # Load the CSV file
    csv_data = pd.read_csv(file_path)
    
    # Add an "account" column
    csv_data['account'] = None
    
    # Iterate through the rows and set the "account" column
    current_account = None
    rows_to_remove = []
    for index, row in csv_data.iterrows():
        first_col_value = str(row.iloc[0])
        if first_col_value in section_headers:
            current_account = first_col_value
            rows_to_remove.append(index)
        else:
            csv_data.at[index, 'account'] = current_account
    
    # Remove the rows with section headers
    csv_data = csv_data.drop(rows_to_remove)
    
    # Save the modified data back to the CSV file
    csv_data.to_csv(file_path, index=False)

# Loop through all files in the csv folder
for filename in os.listdir(csv_folder):
    # Check if the file is a .csv file
    if filename.endswith('.csv'):
        # Construct the full file path
        file_path = os.path.join(csv_folder, filename)
        
        # Process the CSV file
        process_csv_file(file_path)

print("Processing complete!")


Processing complete!


Переносим даты из заголовков в отдельное поле

In [134]:
import os
import re
import pandas as pd

# Define the folder containing the CSV files
csv_folder = 'csv'

# Regular expression pattern to match dates in the filename (dd.mm.yyyy)
date_pattern = re.compile(r'\d{2}\.\d{2}\.\d{4}')

def add_doc_date_column(file_path, date):
    # Load the CSV file
    csv_data = pd.read_csv(file_path)
    
    # Add the "doc_date" column with the extracted date
    csv_data['doc_date'] = date
    
    # Save the modified data back to the CSV file
    csv_data.to_csv(file_path, index=False)

# Loop through all files in the csv folder
for filename in os.listdir(csv_folder):
    # Check if the file is a .csv file
    if filename.endswith('.csv'):
        # Search for the date in the filename
        match = date_pattern.search(filename)
        if match:
            # Extract the date from the filename
            date = match.group(0)
            
            # Construct the full file path
            file_path = os.path.join(csv_folder, filename)
            
            # Add the "doc_date" column to the CSV file
            add_doc_date_column(file_path, date)

print("Date processing complete!")


Date processing complete!


Now extract quarters from filenames

In [135]:
import os
import re
import pandas as pd

# Define the folder containing the CSV files
csv_folder = 'csv'

# Regular expression pattern to match quarter in the filename ({quarter index}кв. {year in YYYY format}г.)
quarter_pattern = re.compile(r'(\d)\s*кв\.\s*(\d{4})г\.')

def add_doc_quarter_column(file_path, quarter):
    # Load the CSV file
    csv_data = pd.read_csv(file_path)
    
    # Add the "doc_quarter" column with the extracted quarter
    csv_data['doc_quarter'] = quarter
    
    # Save the modified data back to the CSV file
    csv_data.to_csv(file_path, index=False)

# Loop through all files in the csv folder
for filename in os.listdir(csv_folder):
    # Check if the file is a .csv file
    if filename.endswith('.csv'):
        # Search for the quarter in the filename
        match = quarter_pattern.search(filename)
        if match:
            # Extract the quarter index and year from the filename
            quarter_index = match.group(1)
            year = match.group(2)
            quarter = f"{year}-Q{quarter_index}"
            
            # Construct the full file path
            file_path = os.path.join(csv_folder, filename)
            
            # Add the "doc_quarter" column to the CSV file
            add_doc_quarter_column(file_path, quarter)

print("Quarter processing complete!")


Quarter processing complete!


Удаляем пустые колонки

In [136]:
import os
import pandas as pd

# Define the folder containing the CSV files
csv_folder = 'csv'

def remove_empty_columns(file_path):
    # Load the CSV file
    csv_data = pd.read_csv(file_path)
    
    # Remove columns that are fully empty or just have the header filled
    cleaned_data = csv_data.dropna(axis=1, how='all')
    
    # Save the modified data back to the CSV file
    cleaned_data.to_csv(file_path, index=False)

# Loop through all files in the csv folder
for filename in os.listdir(csv_folder):
    # Check if the file is a .csv file
    if filename.endswith('.csv'):
        # Construct the full file path
        file_path = os.path.join(csv_folder, filename)
        
        # Remove empty columns from the CSV file
        remove_empty_columns(file_path)

print("Empty columns removal complete!")


Empty columns removal complete!


Переименовываем заголовки полей

In [137]:
import os
import re
import pandas as pd
from collections import defaultdict
from transliterate import translit

# Define the folder containing the CSV files
csv_folder = 'csv'

# Regular expression patterns for field name replacements
quarter_balance_pattern = re.compile(r'Остаток на \d{1,2} \w+ \d{4} г\.')
quarter_turnover_pattern = re.compile(r'Оборот за \d квартал \d{4} г\.')
unnamed_pattern = re.compile(r'Unnamed: \d+')

# Function to transliterate and normalize field names
def normalize_field_name(name):
    # Transliterate from Russian to Latin
    name = translit(name, 'ru', reversed=True)
    # Convert to lowercase and replace spaces with underscores
    name = name.lower().replace(' ', '_')
    # Replace inappropriate symbols
    name = re.sub(r'[^\w_]', '', name)
    return name

# Function to rename fields based on given rules
def rename_fields(df):
    columns = df.columns.tolist()
    new_columns = []
    unnamed_replacements = {}
    
    for col in columns:
        if quarter_balance_pattern.match(col):
            new_columns.append('quarter_balance_number')
        elif quarter_turnover_pattern.match(col):
            new_columns.append('quarter_turnover_number')
        elif unnamed_pattern.match(col):
            new_columns.append(None)  # Placeholder for now
        else:
            new_columns.append(normalize_field_name(col))
    
    # Replace "Unnamed" columns with transliterated nearest non-empty value
    for i, col in enumerate(new_columns):
        if col is None:
            nearest_non_empty = df.iloc[:, i].dropna().iloc[0]
            transliterated = normalize_field_name(nearest_non_empty)
            new_columns[i] = transliterated
    
    # Handle duplicate column names by adding indexes
    counter = defaultdict(int)
    final_columns = []
    for col in new_columns:
        if col in counter:
            counter[col] += 1
            final_columns.append(f"{col}_{counter[col]}")
        else:
            counter[col] = 1
            final_columns.append(col)
    
    df.columns = final_columns
    return df

# Function to process each CSV file
def process_files():
    for filename in os.listdir(csv_folder):
        if filename.endswith('.csv'):
            file_path = os.path.join(csv_folder, filename)
            df = pd.read_csv(file_path)
            df = rename_fields(df)
            df.to_csv(file_path, index=False)

# Execute the processing
process_files()

print("Field renaming and normalization complete!")


Field renaming and normalization complete!


Тюнинг названий полей

In [138]:
import os
import re
import pandas as pd

# Define the folder containing the CSV files
csv_folder = 'csv'

# Regular expression patterns for different purposes
patterns = {
    "quarterly_turnover_balance_101": re.compile(r'Оборотно-сальдовая ведомость по сч\. 101 за \d кв\. \d{4}г\..csv'),
    "quarterly_turnover_balance_21": re.compile(r'Оборотно-сальдовая ведомость по сч\. 21 за \d кв\. \d{4}г\..csv'),
    "quarterly_turnover_balance_105": re.compile(r'Оборотно-сальдовая ведомость по сч\. 105 за \d кв\. \d{4}г\..csv'),
    "quarterly_turnover_101": re.compile(r'Оборотная ведомость по сч\. 101 за \d кв\. \d{4}г\..csv'),
    "quarterly_turnover_21": re.compile(r'Оборотная ведомость по сч\. 21 за \d кв\. \d{4}г\..csv'),
    "quarterly_turnover_105": re.compile(r'Оборотная ведомость по сч\. 105 за \d кв\. \d{4}г\..csv'),
    "date_balance_statement_101": re.compile(r'Ведомость остатков на \d{2}\.\d{2}\.\d{4}(г\.)? \(сч\. 101\)\.csv'),
    "date_balance_statement_21": re.compile(r'Ведомость остатков на \d{2}\.\d{2}\.\d{4}(г\.)? \(сч\. 21\)\.csv'),
    "date_balance_statement_105": re.compile(r'Ведомость остатков на \d{2}\.\d{2}\.\d{4}(г\.)? \(сч\. 105\)\.csv')
}

# Function to classify files based on their filename
def classify_files():
    classified_files = {
        "quarterly_turnover_balance_101": [],
        "quarterly_turnover_balance_21": [],
        "quarterly_turnover_balance_105": [],
        "quarterly_turnover_101": [],
        "quarterly_turnover_21": [],
        "quarterly_turnover_105": [],
        "date_balance_statement_101": [],
        "date_balance_statement_21": [],
        "date_balance_statement_105": []
    }
    
    for filename in os.listdir(csv_folder):
        # Check if the file is a .csv file
        if filename.endswith('.csv'):
            for key, pattern in patterns.items():
                if pattern.match(filename):
                    classified_files[key].append(filename)
                    break
    
    return classified_files

# Function to analyze CSV files and generate a translation map template
def generate_translation_map():
    classified_files = classify_files()
    translation_map = {}

    for purpose, files in classified_files.items():
        field_names = set()
        for file in files:
            file_path = os.path.join(csv_folder, file)
            df = pd.read_csv(file_path)
            field_names.update(df.columns.tolist())
        
        translation_map[purpose] = [{old_name: old_name} for old_name in sorted(field_names)]

    return translation_map

# Generate the translation map
translation_map = generate_translation_map()

# Print the translation map
for purpose, fields in translation_map.items():
    print(f"{purpose}:")
    for field in fields:
        print(f"    {field},")

print("Translation map template generated!")


quarterly_turnover_balance_101:
    {'account': 'account'},
    {'doc_quarter': 'doc_quarter'},
    {'dolzhnost': 'dolzhnost'},
    {'gosudarstvennoe_kazennoe_uchrezhdenie_goroda_moskvy_informatsionnyj_gorod': 'gosudarstvennoe_kazennoe_uchrezhdenie_goroda_moskvy_informatsionnyj_gorod'},
    {'kfo_ravno_1_i_subkonto2_ravno_kuznetsov_m_s__': 'kfo_ravno_1_i_subkonto2_ravno_kuznetsov_m_s__'},
    {'kredit': 'kredit'},
    {'kredit_2': 'kredit_2'},
    {'kredit_3': 'kredit_3'},
    {'oboroty_za_period': 'oboroty_za_period'},
    {'podpis': 'podpis'},
    {'pokazateli': 'pokazateli'},
    {'produkt_v_interesah_kotorogo_osuschestvljaetsja_zakupka': 'produkt_v_interesah_kotorogo_osuschestvljaetsja_zakupka'},
    {'saldo_na_konets_perioda': 'saldo_na_konets_perioda'},
    {'saldo_na_nachalo_perioda': 'saldo_na_nachalo_perioda'},
quarterly_turnover_balance_21:
    {'account': 'account'},
    {'doc_quarter': 'doc_quarter'},
    {'dolzhnost': 'dolzhnost'},
    {'gosudarstvennoe_kazennoe_uchrezhden

In [139]:
translation_map = {
    "quarterly_turnover_balance_101": [
        {'doc_quarter': 'doc_quarter'},
        {'dolzhnost': ''},
        {'gosudarstvennoe_kazennoe_uchrezhdenie_goroda_moskvy_informatsionnyj_gorod': 'name'},
        {'kfo_ravno_1_i_subkonto2_ravno_kuznetsov_m_s__': ''},
        {'kredit': 'balance_start_credit'},
        {'kredit_2': 'turnover_credit'},
        {'kredit_3': 'balance_end_credit'},
        {'oboroty_za_period': 'turnover_for_period'},
        {'podpis': ''},
        {'pokazateli': 'indicators'},
        {'produkt_v_interesah_kotorogo_osuschestvljaetsja_zakupka': 'product_of_interest_for_procurement'},
        {'saldo_na_konets_perioda': 'balance_end_period'},
        {'saldo_na_nachalo_perioda': 'balance_start_period'},
    ],
    "quarterly_turnover_balance_21": [
        {'account': 'account'},
        {'doc_quarter': 'doc_quarter'},
        {'dolzhnost': ''},
        {'gosudarstvennoe_kazennoe_uchrezhdenie_goroda_moskvy_informatsionnyj_gorod': 'name'},
        {'kfo_ravno_1_i_subkonto2_ravno_kuznetsov_m_s__': ''},
        {'kredit': 'balance_start_credit'},
        {'kredit_2': 'turnover_credit'},
        {'kredit_3': 'balance_end_credit'},
        {'oboroty_za_period': 'turnover_for_period'},
        {'podpis': ''},
        {'pokazateli': 'indicators'},
        {'produkt_v_interesah_kotorogo_osuschestvljaetsja_zakupka': 'product_of_interest_for_procurement'},
        {'saldo_na_konets_perioda': 'balance_end_period'},
        {'saldo_na_nachalo_perioda': 'balance_start_period'},
    ],
    "quarterly_turnover_balance_105": [
        # Assuming similar fields as other quarterly_turnover_balance, please update if there are specific fields for 105
        {'doc_quarter': 'doc_quarter'},
        {'dolzhnost': ''},
        {'gosudarstvennoe_kazennoe_uchrezhdenie_goroda_moskvy_informatsionnyj_gorod': 'name'},
        {'kfo_ravno_1_i_subkonto2_ravno_kuznetsov_m_s__': ''},
        {'kredit': 'balance_start_credit'},
        {'kredit_2': 'turnover_credit'},
        {'kredit_3': 'balance_end_credit'},
        {'oboroty_za_period': 'turnover_for_period'},
        {'podpis': ''},
        {'pokazateli': 'indicators'},
        {'produkt_v_interesah_kotorogo_osuschestvljaetsja_zakupka': 'product_of_interest_for_procurement'},
        {'saldo_na_konets_perioda': 'balance_end_period'},
        {'saldo_na_nachalo_perioda': 'balance_start_period'},
    ],
    "quarterly_turnover_101": [
        {'_pp': 'pp'},
        {'doc_quarter': 'doc_quarter'},
        {'edinitsa_izmerenija': 'unit_of_measurement'},
        {'inventarnyj_nomenklaturnyjnomer': 'inventory_item_number'},
        {'kod_spravochnika': 'reference_code'},
        {'kredit': 'quarter_turnover_credit_number'},
        {'naimenovanie_nefinansovogo_aktiva': 'non_financial_asset_name'},
        {'quarter_balance_number': 'quarter_balance_number_begin'},
        {'quarter_balance_number_2': 'quarter_balance_number_end'},
        {'quarter_turnover_number': 'quarter_turnover_debit_number'},
        {'summa': 'quarter_balance_amount_begin'},
        {'summa_2': 'quarter_turnover_debit_amount'},
        {'summa_3': 'quarter_turnover_credit_amount'},
        {'summa_4': 'quarter_balance_amount_end'},
    ],
    "quarterly_turnover_21": [
        {'_pp': 'pp'},
        {'doc_quarter': 'doc_quarter'},
        {'edinitsa_izmerenija': 'unit_of_measurement'},
        {'inventarnyj_nomenklaturnyjnomer': 'inventory_item_number'},
        {'kod_spravochnika': 'reference_code'},
        {'kredit': 'quarter_turnover_credit_number'},
        {'naimenovanie_nefinansovogo_aktiva': 'non_financial_asset_name'},
        {'quarter_balance_number': 'quarter_balance_number_begin'},
        {'quarter_balance_number_2': 'quarter_balance_number_end'},
        {'quarter_turnover_number': 'quarter_turnover_debit_number'},
        {'summa': 'quarter_balance_amount_begin'},
        {'summa_2': 'quarter_turnover_debit_amount'},
        {'summa_3': 'quarter_turnover_credit_amount'},
        {'summa_4': 'quarter_balance_amount_end'},
    ],
    "quarterly_turnover_105": [
        {'_pp': 'pp'},
        {'doc_quarter': 'doc_quarter'},
        {'edinitsa_izmerenija': 'unit_of_measurement'},
        {'inventarnyj_nomenklaturnyjnomer': 'inventory_item_number'},
        {'kod_spravochnika': 'reference_code'},
        {'kredit': 'quarter_turnover_credit_number'},
        {'naimenovanie_nefinansovogo_aktiva': 'non_financial_asset_name'},
        {'quarter_balance_number': 'quarter_balance_number_begin'},
        {'quarter_balance_number_2': 'quarter_balance_number_end'},
        {'quarter_turnover_number': 'quarter_turnover_debit_number'},
        {'summa': 'quarter_balance_amount_begin'},
        {'summa_2': 'quarter_turnover_debit_amount'},
        {'summa_3': 'quarter_turnover_credit_amount'},
        {'summa_4': 'quarter_balance_amount_end'},
    ],
    "date_balance_statement_101": [
        {'amortizatsionnaja_gruppa': 'depreciation_group'},
        {'balansovaja_stoimost': 'balance_value'},
        {'data_prinjatija_k_uchetu': 'accounting_date'},
        {'doc_date': 'doc_date'},
        {'dolzhnost': ''},
        {'inventarnyj_nomer': 'inventory_number'},
        {'iznos_': 'wear'},
        {'kolichestvo': 'quantity'},
        {'mes_norma_iznosa_': 'monthly_wear_norm'},
        {'okof': 'okof'},
        {'osnovnoe_sredstvo': 'fixed_asset'},
        {'ostatochnaja_stoimost': 'residual_value'},
        {'podpis': ''},
        {'rasshifrovka_podpisi': ''},
        {'sostojanie': 'condition'},
        {'sposob_nachislenija_amortizatsii': 'depreciation_method'},
        {'srok_poleznogo_ispolzovanija': 'useful_life'},
        {'summa_amortizatsii': 'depreciation_amount'},
        {'vedomost_ostatkov_os_nma_npa': 'balance_statement_os_nma_npa'},
    ],
    "date_balance_statement_21": [
        {'amortizatsionnaja_gruppa': 'depreciation_group'},
        {'balansovaja_stoimost': 'balance_value'},
        {'data_prinjatija_k_uchetu': 'accounting_date'},
        {'doc_date': 'doc_date'},
        {'dolzhnost': ''},
        {'inventarnyj_nomer': 'inventory_number'},
        {'iznos_': 'wear'},
        {'kolichestvo': 'quantity'},
        {'mes_norma_iznosa_': 'monthly_wear_norm'},
        {'okof': 'okof'},
        {'osnovnoe_sredstvo': 'fixed_asset'},
        {'ostatochnaja_stoimost': 'residual_value'},
        {'podpis': ''},
        {'rasshifrovka_podpisi': ''},
        {'sostojanie': 'condition'},
        {'sposob_nachislenija_amortizatsii': 'depreciation_method'},
        {'srok_poleznogo_ispolzovanija': 'useful_life'},
        {'summa_amortizatsii': 'depreciation_amount'},
        {'vedomost_ostatkov_os_nma_npa': 'balance_statement_os_nma_npa'},
    ],
    "date_balance_statement_105": [
        {'doc_date': 'doc_date'},
        {'kolichestvo': 'quantity'},
        {'mol': 'responsible_person'},
        {'summa': 'amount'},
        {'tsena': 'price'},
    ]
}

# Print the translation map
for purpose, fields in translation_map.items():
    print(f"{purpose}:")
    for field in fields:
        print(f"    {field},")


quarterly_turnover_balance_101:
    {'doc_quarter': 'doc_quarter'},
    {'dolzhnost': ''},
    {'gosudarstvennoe_kazennoe_uchrezhdenie_goroda_moskvy_informatsionnyj_gorod': 'name'},
    {'kfo_ravno_1_i_subkonto2_ravno_kuznetsov_m_s__': ''},
    {'kredit': 'balance_start_credit'},
    {'kredit_2': 'turnover_credit'},
    {'kredit_3': 'balance_end_credit'},
    {'oboroty_za_period': 'turnover_for_period'},
    {'podpis': ''},
    {'pokazateli': 'indicators'},
    {'produkt_v_interesah_kotorogo_osuschestvljaetsja_zakupka': 'product_of_interest_for_procurement'},
    {'saldo_na_konets_perioda': 'balance_end_period'},
    {'saldo_na_nachalo_perioda': 'balance_start_period'},
quarterly_turnover_balance_21:
    {'account': 'account'},
    {'doc_quarter': 'doc_quarter'},
    {'dolzhnost': ''},
    {'gosudarstvennoe_kazennoe_uchrezhdenie_goroda_moskvy_informatsionnyj_gorod': 'name'},
    {'kfo_ravno_1_i_subkonto2_ravno_kuznetsov_m_s__': ''},
    {'kredit': 'balance_start_credit'},
    {'kredit_

In [140]:
import os
import pandas as pd

# Define the folder containing the CSV files
csv_folder = 'csv'

# Provided translation map
translation_map = {
    "quarterly_turnover_balance_101": [
        {'doc_quarter': 'doc_quarter'},
        {'dolzhnost': ''},
        {'gosudarstvennoe_kazennoe_uchrezhdenie_goroda_moskvy_informatsionnyj_gorod': 'name'},
        {'kfo_ravno_1_i_subkonto2_ravno_kuznetsov_m_s__': ''},
        {'kredit': 'balance_start_credit'},
        {'kredit_2': 'turnover_credit'},
        {'kredit_3': 'balance_end_credit'},
        {'oboroty_za_period': 'turnover_for_period'},
        {'podpis': ''},
        {'pokazateli': 'indicators'},
        {'produkt_v_interesah_kotorogo_osuschestvljaetsja_zakupka': 'product_of_interest_for_procurement'},
        {'saldo_na_konets_perioda': 'balance_end_period'},
        {'saldo_na_nachalo_perioda': 'balance_start_period'},
    ],
    "quarterly_turnover_balance_21": [
        {'account': 'account'},
        {'doc_quarter': 'doc_quarter'},
        {'dolzhnost': ''},
        {'gosudarstvennoe_kazennoe_uchrezhdenie_goroda_moskvy_informatsionnyj_gorod': 'name'},
        {'kfo_ravno_1_i_subkonto2_ravno_kuznetsov_m_s__': ''},
        {'kredit': 'balance_start_credit'},
        {'kredit_2': 'turnover_credit'},
        {'kredit_3': 'balance_end_credit'},
        {'oboroty_za_period': 'turnover_for_period'},
        {'podpis': ''},
        {'pokazateli': 'indicators'},
        {'produkt_v_interesah_kotorogo_osuschestvljaetsja_zakupka': 'product_of_interest_for_procurement'},
        {'saldo_na_konets_perioda': 'balance_end_period'},
        {'saldo_na_nachalo_perioda': 'balance_start_period'},
    ],
    "quarterly_turnover_balance_105": [
        {'doc_quarter': 'doc_quarter'},
        {'dolzhnost': ''},
        {'gosudarstvennoe_kazennoe_uchrezhdenie_goroda_moskvy_informatsionnyj_gorod': 'name'},
        {'kfo_ravno_1_i_subkonto2_ravno_kuznetsov_m_s__': ''},
        {'kredit': 'balance_start_credit'},
        {'kredit_2': 'turnover_credit'},
        {'kredit_3': 'balance_end_credit'},
        {'oboroty_za_period': 'turnover_for_period'},
        {'podpis': ''},
        {'pokazateli': 'indicators'},
        {'produkt_v_interesah_kotorogo_osuschestvljaetsja_zakupka': 'product_of_interest_for_procurement'},
        {'saldo_na_konets_perioda': 'balance_end_period'},
        {'saldo_na_nachalo_perioda': 'balance_start_period'},
    ],
    "quarterly_turnover_101": [
        {'_pp': 'pp'},
        {'doc_quarter': 'doc_quarter'},
        {'edinitsa_izmerenija': 'unit_of_measurement'},
        {'inventarnyj_nomenklaturnyjnomer': 'inventory_item_number'},
        {'kod_spravochnika': 'reference_code'},
        {'kredit': 'quarter_turnover_credit_number'},
        {'naimenovanie_nefinansovogo_aktiva': 'non_financial_asset_name'},
        {'quarter_balance_number': 'quarter_balance_number_begin'},
        {'quarter_balance_number_2': 'quarter_balance_number_end'},
        {'quarter_turnover_number': 'quarter_turnover_debit_number'},
        {'summa': 'quarter_balance_amount_begin'},
        {'summa_2': 'quarter_turnover_debit_amount'},
        {'summa_3': 'quarter_turnover_credit_amount'},
        {'summa_4': 'quarter_balance_amount_end'},
    ],
    "quarterly_turnover_21": [
        {'_pp': 'pp'},
        {'doc_quarter': 'doc_quarter'},
        {'edinitsa_izmerenija': 'unit_of_measurement'},
        {'inventarnyj_nomenklaturnyjnomer': 'inventory_item_number'},
        {'kod_spravochnika': 'reference_code'},
        {'kredit': 'quarter_turnover_credit_number'},
        {'naimenovanie_nefinansovogo_aktiva': 'non_financial_asset_name'},
        {'quarter_balance_number': 'quarter_balance_number_begin'},
        {'quarter_balance_number_2': 'quarter_balance_number_end'},
        {'quarter_turnover_number': 'quarter_turnover_debit_number'},
        {'summa': 'quarter_balance_amount_begin'},
        {'summa_2': 'quarter_turnover_debit_amount'},
        {'summa_3': 'quarter_turnover_credit_amount'},
        {'summa_4': 'quarter_balance_amount_end'},
    ],
    "quarterly_turnover_105": [
        {'_pp': 'pp'},
        {'doc_quarter': 'doc_quarter'},
        {'edinitsa_izmerenija': 'unit_of_measurement'},
        {'inventarnyj_nomenklaturnyjnomer': 'inventory_item_number'},
        {'kod_spravochnika': 'reference_code'},
        {'kredit': 'quarter_turnover_credit_number'},
        {'naimenovanie_nefinansovogo_aktiva': 'non_financial_asset_name'},
        {'quarter_balance_number': 'quarter_balance_number_begin'},
        {'quarter_balance_number_2': 'quarter_balance_number_end'},
        {'quarter_turnover_number': 'quarter_turnover_debit_number'},
        {'summa': 'quarter_balance_amount_begin'},
        {'summa_2': 'quarter_turnover_debit_amount'},
        {'summa_3': 'quarter_turnover_credit_amount'},
        {'summa_4': 'quarter_balance_amount_end'},
    ],
    "date_balance_statement_101": [
        {'amortizatsionnaja_gruppa': 'depreciation_group'},
        {'balansovaja_stoimost': 'balance_value'},
        {'data_prinjatija_k_uchetu': 'accounting_date'},
        {'doc_date': 'doc_date'},
        {'dolzhnost': ''},
        {'inventarnyj_nomer': 'inventory_number'},
        {'iznos_': 'wear'},
        {'kolichestvo': 'quantity'},
        {'mes_norma_iznosa_': 'monthly_wear_norm'},
        {'okof': 'okof'},
        {'osnovnoe_sredstvo': 'fixed_asset'},
        {'ostatochnaja_stoimost': 'residual_value'},
        {'podpis': 'signature'},
        {'rasshifrovka_podpisi': ''},
        {'sostojanie': 'condition'},
        {'sposob_nachislenija_amortizatsii': 'depreciation_method'},
        {'srok_poleznogo_ispolzovanija': 'useful_life'},
        {'summa_amortizatsii': 'depreciation_amount'},
        {'vedomost_ostatkov_os_nma_npa': 'balance_statement_os_nma_npa'},
    ],
    "date_balance_statement_21": [
        {'amortizatsionnaja_gruppa': 'depreciation_group'},
        {'balansovaja_stoimost': 'balance_value'},
        {'data_prinjatija_k_uchetu': 'accounting_date'},
        {'doc_date': 'doc_date'},
        {'dolzhnost': ''},
        {'inventarnyj_nomer': 'inventory_number'},
        {'iznos_': 'wear'},
        {'kolichestvo': 'quantity'},
        {'mes_norma_iznosa_': 'monthly_wear_norm'},
        {'okof': 'okof'},
        {'osnovnoe_sredstvo': 'fixed_asset'},
        {'ostatochnaja_stoimost': 'residual_value'},
        {'podpis': 'signature'},
        {'rasshifrovka_podpиси': 'signature_decryption'},
        {'sostojanie': 'condition'},
        {'sposob_nachislenija_amortizatsii': 'depreciation_method'},
        {'srok_poleznogo_ispolzovanija': 'useful_life'},
        {'summa_amortizatsii': 'depreciation_amount'},
        {'vedomost_ostatkov_os_nma_npa': 'balance_statement_os_nma_npa'},
    ],
    "date_balance_statement_105": [
        {'doc_date': 'doc_date'},
        {'kolichestvo': 'quantity'},
        {'mol': 'responsible_person'},
        {'summa': 'amount'},
        {'tsena': 'price'},
    ]
}

# Function to classify files based on their filename
def classify_files():
    classified_files = {
        "quarterly_turnover_balance_101": [],
        "quarterly_turnover_balance_21": [],
        "quarterly_turnover_balance_105": [],
        "quarterly_turnover_101": [],
        "quarterly_turnover_21": [],
        "quarterly_turnover_105": [],
        "date_balance_statement_101": [],
        "date_balance_statement_21": [],
        "date_balance_statement_105": []
    }
    
    patterns = {
        "quarterly_turnover_balance_101": re.compile(r'Оборотно-сальдовая ведомость по сч\. 101 за \d кв\. \d{4}г\..csv'),
        "quarterly_turnover_balance_21": re.compile(r'Оборотно-сальдовая ведомость по сч\. 21 за \d кв\. \d{4}г\..csv'),
        "quarterly_turnover_balance_105": re.compile(r'Оборотно-сальдовая ведомость по сч\. 105 за \d кв\. \d{4}г\..csv'),
        "quarterly_turnover_101": re.compile(r'Оборотная ведомость по сч\. 101 за \d кв\. \d{4}г\..csv'),
        "quarterly_turnover_21": re.compile(r'Оборотная ведомость по сч\. 21 за \d кв\. \d{4}г\..csv'),
        "quarterly_turnover_105": re.compile(r'Оборотная ведомость по сч\. 105 за \d кв\. \d{4}г\..csv'),
        "date_balance_statement_101": re.compile(r'Ведомость остатков на \d{2}\.\d{2}\.\d{4}г?\.?\s*\(сч\.?\s*101\)\.csv'),
        "date_balance_statement_21": re.compile(r'Ведомость остатков на \d{2}\.\d{2}\.\d{4}(г\.)? \(сч\. 21\)\.csv'),
        "date_balance_statement_105": re.compile(r'Ведомость остатков на \d{2}\.\d{2}\.\d{4}(г\.)? \(сч\. 105\)\.csv')
    }
    
    for filename in os.listdir(csv_folder):
        if filename.endswith('.csv'):
            for key, pattern in patterns.items():
                if pattern.match(filename):
                    classified_files[key].append(filename)
                    break
    
    return classified_files

# Function to rename fields in a DataFrame according to the translation map
def rename_fields(df, translation_map):
    for translation in translation_map:
        old_name, new_name = list(translation.items())[0]
        if new_name:
            df.rename(columns={old_name: new_name}, inplace=True)
        else:
            df.drop(columns=[old_name], inplace=True)
    return df

# Function to process and rename fields in each file based on its classification
def process_files():
    classified_files = classify_files()
    
    for purpose, files in classified_files.items():
        for file in files:
            file_path = os.path.join(csv_folder, file)
            df = pd.read_csv(file_path)
            df = rename_fields(df, translation_map[purpose])
            df.to_csv(file_path, index=False)

# Execute the processing
process_files()

print("Field renaming and column removal complete!")


Field renaming and column removal complete!


Убираем двойные ряды сумма/количество для Оборотно-сальдовая ведомость - добавляем колонки

In [141]:
import os
import pandas as pd

# Define the folder containing the CSV files
csv_folder = 'csv'

# Columns to rename and add new columns
columns_to_process = [
    'balance_start_period',
    'balance_start_credit',
    'turnover_for_period',
    'turnover_credit',
    'balance_end_period',
    'balance_end_credit'
]

# Function to process each CSV file
def process_file(file_path):
    df = pd.read_csv(file_path)

    for column in columns_to_process:
        if column in df.columns:
            # Rename the column
            new_column_name = f"{column}_amount"
            df.rename(columns={column: new_column_name}, inplace=True)
            
            # Insert the new column with name "{oldname}_number" after the renamed column
            new_column_index = df.columns.get_loc(new_column_name) + 1
            df.insert(new_column_index, f"{column}_number", None)  # Initialize with None or any default value

    df.to_csv(file_path, index=False)

# Loop through all files in the csv folder
for filename in os.listdir(csv_folder):
    if filename.startswith("Оборотно-сальдовая ведомость") and filename.endswith('.csv'):
        file_path = os.path.join(csv_folder, filename)
        process_file(file_path)

print("Column renaming and addition complete!")


Column renaming and addition complete!


Добавляем значения из вторых рядов в первые

In [142]:
import os
import pandas as pd

# Define the folder containing the CSV files
csv_folder = 'csv'

# Columns to process
columns_to_process = [
    'balance_start_period',
    'balance_start_credit',
    'turnover_for_period',
    'turnover_credit',
    'balance_end_period',
    'balance_end_credit'
]

# Function to process each CSV file
def process_file(file_path):
    df = pd.read_csv(file_path)
    
    for prefix in columns_to_process:
        amount_col = f"{prefix}_amount"
        number_col = f"{prefix}_number"
        
        # Ensure both columns exist before processing
        if amount_col in df.columns and number_col in df.columns:
            for i in range(1, len(df)):
                if df.loc[i, 'indicators'] == 'Кол.':
                    df.loc[i-1, number_col] = df.loc[i, amount_col]

    df.to_csv(file_path, index=False)

# Loop through all files in the csv folder
for filename in os.listdir(csv_folder):
    if filename.startswith("Оборотно-сальдовая ведомость") and filename.endswith('.csv'):
        file_path = os.path.join(csv_folder, filename)
        process_file(file_path)

print("Column value copying complete!")


/var/folders/sv/qy8vlp51007g2k72ybwrhhqw0000gp/T/ipykernel_21930/3546788753.py:29: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '5436' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[i-1, number_col] = df.loc[i, amount_col]
/var/folders/sv/qy8vlp51007g2k72ybwrhhqw0000gp/T/ipykernel_21930/3546788753.py:29: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '22' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[i-1, number_col] = df.loc[i, amount_col]
/var/folders/sv/qy8vlp51007g2k72ybwrhhqw0000gp/T/ipykernel_21930/3546788753.py:29: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '5458' has dtype incompatible with float64, please explicitly cast to a compa

Column value copying complete!


Убираем Кол. и Колонку indicators

In [143]:
import os
import pandas as pd

# Define the folder containing the CSV files
csv_folder = 'csv'

# Function to process each CSV file
def process_file(file_path):
    df = pd.read_csv(file_path)
    
    # Remove rows where indicators == "Кол."
    df = df[df['indicators'] != 'Кол.']
    
    # Remove the indicators column
    if 'indicators' in df.columns:
        df.drop(columns=['indicators'], inplace=True)
    
    df.to_csv(file_path, index=False)

# Loop through all files in the csv folder
for filename in os.listdir(csv_folder):
    if filename.startswith("Оборотно-сальдовая ведомость") and filename.endswith('.csv'):
        file_path = os.path.join(csv_folder, filename)
        process_file(file_path)

print("Rows and column removal complete!")


Rows and column removal complete!


Second row section header process

In [144]:
import os
import pandas as pd

# Define the folder containing the CSV files
csv_folder = 'csv'

# Prefixes to match section headers
section_prefixes = ("105.34", "105.35", "105.36")

# Function to process each CSV file
def process_file(file_path):
    print(f"Processing file: {file_path}")
    df = pd.read_csv(file_path)
    
    # Add account column if it doesn't exist
    if 'account' not in df.columns:
        df['account'] = None
    print("Columns in the dataframe:", df.columns.tolist())
    
    current_account = None
    rows_to_remove = []

    for index, row in df.iterrows():
        if isinstance(row.iloc[1], str) and row.iloc[1].startswith(section_prefixes):  # Use the second column (index 1)
            print(f"Found section header at row {index}: {row.iloc[1]}")
            current_account = row.iloc[1]
            rows_to_remove.append(index)
        else:
            df.at[index, 'account'] = current_account
    
    print("Rows to remove:", rows_to_remove)
    print("Current account values:", df['account'].unique())
    
    # Remove the rows with section headers
    df.drop(rows_to_remove, inplace=True)
    
    df.to_csv(file_path, index=False)
    print(f"Finished processing file: {file_path}")

# Loop through all files in the csv folder
for filename in os.listdir(csv_folder):
    if filename.endswith('.csv'):
        file_path = os.path.join(csv_folder, filename)
        process_file(file_path)

print("Account column processing complete!")


Processing file: csv/Ведомость остатков на 31.03.2022г. (сч. 21).csv
Columns in the dataframe: ['balance_statement_os_nma_npa', 'fixed_asset', 'signature', 'rasshifrovka_podpisi', 'inventory_number', 'okof', 'depreciation_group', 'depreciation_method', 'accounting_date', 'condition', 'useful_life', 'monthly_wear_norm', 'wear', 'balance_value', 'quantity', 'depreciation_amount', 'residual_value', 'account', 'doc_date']
Rows to remove: []
Current account values: [None]
Finished processing file: csv/Ведомость остатков на 31.03.2022г. (сч. 21).csv
Processing file: csv/Ведомость остатков на 30.09.2022г. (сч. 21).csv
Columns in the dataframe: ['balance_statement_os_nma_npa', 'fixed_asset', 'signature', 'rasshifrovka_podpisi', 'inventory_number', 'okof', 'depreciation_group', 'depreciation_method', 'accounting_date', 'condition', 'useful_life', 'monthly_wear_norm', 'wear', 'balance_value', 'quantity', 'depreciation_amount', 'residual_value', 'account', 'doc_date']
Rows to remove: []
Current a

getting doc account

In [145]:
import os
import re
import pandas as pd

# Define the folder containing the CSV files
csv_folder = 'csv'

# Function to extract account number from file name
def extract_account_number(filename):
    match = re.search(r'сч\. (\d+)', filename)
    return match.group(1) if match else None

# Function to process each CSV file
def process_file(file_path, account_number):
    print(f"Processing file: {file_path}")
    df = pd.read_csv(file_path)
    
    # Add doc_account column with the extracted account number
    df['doc_account'] = account_number
    
    df.to_csv(file_path, index=False)
    print(f"Finished processing file: {file_path}")

# Loop through all files in the csv folder
for filename in os.listdir(csv_folder):
    if filename.endswith('.csv'):
        account_number = extract_account_number(filename)
        if account_number:
            file_path = os.path.join(csv_folder, filename)
            process_file(file_path, account_number)

print("Account number column processing complete!")


Processing file: csv/Ведомость остатков на 31.03.2022г. (сч. 21).csv
Finished processing file: csv/Ведомость остатков на 31.03.2022г. (сч. 21).csv
Processing file: csv/Ведомость остатков на 30.09.2022г. (сч. 21).csv
Finished processing file: csv/Ведомость остатков на 30.09.2022г. (сч. 21).csv
Processing file: csv/Ведомость остатков на 30.06.2022г. (сч. 101).csv
Finished processing file: csv/Ведомость остатков на 30.06.2022г. (сч. 101).csv
Processing file: csv/Оборотная ведомость по сч. 105 за 1 кв. 2022г..csv
Finished processing file: csv/Оборотная ведомость по сч. 105 за 1 кв. 2022г..csv
Processing file: csv/Ведомость остатков на 31.03.2022г.(сч. 101).csv
Finished processing file: csv/Ведомость остатков на 31.03.2022г.(сч. 101).csv
Processing file: csv/Ведомость остатков на 31.03.2022 (сч. 105).csv
Finished processing file: csv/Ведомость остатков на 31.03.2022 (сч. 105).csv
Processing file: csv/Ведомость остатков на 30.09.2022 (сч. 105).csv
Finished processing file: csv/Ведомость оста

Cleaning empty rows, totals, and section headers

In [146]:
import os
import pandas as pd

# Define the folder containing the CSV files
csv_folder = 'csv'

# Define the strings and conditions for rows to be removed
strings_to_remove = [
    "Ведомость остатков ОС, НМА, НПА", "Счет", "КФО", "КПС", "ЦМО.Сотрудник", 
    "ЦМО.Место хранения", "№ п/п", "Кузнецов Максим Сергеевич", "04100000000000000", 
    "Номенклатура", "Кузнецов М. С. -", "Единица измерения: рубль (код по ОКЕИ 383)", 
    "Отбор:", "Счет", "Основные средства", "ЦМО", "21.30", "Итого", "Исполнитель", 
    "Главный бухгалтер", "(уполномоченное лицо)", "Руководитель", "(уполномоченное лицо)", 
    "Ответственный:","Место хранения"
]

def remove_unwanted_rows(file_path):
    print(f"Processing file: {file_path}")
    try:
        df = pd.read_csv(file_path)

        # Condition 1: Remove rows where the first column is empty
        initial_shape = df.shape
        df = df.dropna(subset=[df.columns[0]])
        print(f"Removed {initial_shape[0] - df.shape[0]} rows with empty first column.")

        # Condition 2: Remove rows with unwanted strings in the first column
        for string in strings_to_remove:
            initial_shape = df.shape
            df = df[~df[df.columns[0]].astype(str).str.contains(string, na=False)]
            print(f"Removed {initial_shape[0] - df.shape[0]} rows containing '{string}' in the first column.")

        # Condition 3: Remove rows where first column starts with "Оборотно-сальдовая ведомость по счету"
        initial_shape = df.shape
        df = df[~df[df.columns[0]].astype(str).str.startswith("Оборотно-сальдовая ведомость по счету", na=False)]
        print(f"Removed {initial_shape[0] - df.shape[0]} rows starting with 'Оборотно-сальдовая ведомость по счету'.")

        # Condition 4: Remove rows where second column is empty and first column contains specific patterns
        patterns_to_remove = ["1", "21", "101", "101.00", "101.30"]
        initial_shape = df.shape
        for pattern in patterns_to_remove:
            df = df[~((df[df.columns[0]].astype(str).str.contains(pattern, na=False)) & (df[df.columns[1]].isna()))]
        print(f"Removed {initial_shape[0] - df.shape[0]} rows with specific patterns in first column and empty second column.")

        # Save the cleaned DataFrame back to the CSV
        df.to_csv(file_path, index=False)
        print(f"Finished processing file: {file_path}\n")
    except Exception as e:
        print(f"An error occurred while processing {file_path}: {e}")

# Loop through all files in the csv folder
for filename in os.listdir(csv_folder):
    if filename.endswith('.csv'):
        file_path = os.path.join(csv_folder, filename)
        remove_unwanted_rows(file_path)

print("Rows removal process complete!")


Processing file: csv/Ведомость остатков на 31.03.2022г. (сч. 21).csv
Removed 11 rows with empty first column.
Removed 1 rows containing 'Ведомость остатков ОС, НМА, НПА' in the first column.
Removed 1 rows containing 'Счет' in the first column.
Removed 1 rows containing 'КФО' in the first column.
Removed 1 rows containing 'КПС' in the first column.
Removed 1 rows containing 'ЦМО.Сотрудник' in the first column.
Removed 1 rows containing 'ЦМО.Место хранения' in the first column.
Removed 1 rows containing '№ п/п' in the first column.
Removed 3 rows containing 'Кузнецов Максим Сергеевич' in the first column.
Removed 0 rows containing '04100000000000000' in the first column.
Removed 0 rows containing 'Номенклатура' in the first column.
Removed 0 rows containing 'Кузнецов М. С. -' in the first column.
Removed 0 rows containing 'Единица измерения: рубль (код по ОКЕИ 383)' in the first column.
Removed 0 rows containing 'Отбор:' in the first column.
Removed 0 rows containing 'Счет' in the first

/var/folders/sv/qy8vlp51007g2k72ybwrhhqw0000gp/T/ipykernel_21930/2183993706.py:30: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df = df[~df[df.columns[0]].astype(str).str.contains(string, na=False)]
/var/folders/sv/qy8vlp51007g2k72ybwrhhqw0000gp/T/ipykernel_21930/2183993706.py:30: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df = df[~df[df.columns[0]].astype(str).str.contains(string, na=False)]
/var/folders/sv/qy8vlp51007g2k72ybwrhhqw0000gp/T/ipykernel_21930/2183993706.py:30: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df = df[~df[df.columns[0]].astype(str).str.contains(string, na=False)]
/var/folders/sv/qy8vlp51007g2k72ybwrhhqw0000gp/T/ipykernel_21930/2183993706.py:30: UserWarning: This pattern is interpreted as a regu

Removed 0 rows starting with 'Оборотно-сальдовая ведомость по счету'.
Removed 3 rows with specific patterns in first column and empty second column.
Finished processing file: csv/Ведомость остатков на 30.06.2022г. (сч. 101).csv

Processing file: csv/Оборотная ведомость по сч. 105 за 1 кв. 2022г..csv
Removed 5 rows with empty first column.
Removed 0 rows containing 'Ведомость остатков ОС, НМА, НПА' in the first column.
Removed 0 rows containing 'Счет' in the first column.
Removed 0 rows containing 'КФО' in the first column.
Removed 0 rows containing 'КПС' in the first column.
Removed 0 rows containing 'ЦМО.Сотрудник' in the first column.
Removed 0 rows containing 'ЦМО.Место хранения' in the first column.
Removed 0 rows containing '№ п/п' in the first column.
Removed 0 rows containing 'Кузнецов Максим Сергеевич' in the first column.
Removed 0 rows containing '04100000000000000' in the first column.
Removed 0 rows containing 'Номенклатура' in the first column.
Removed 0 rows containing 'К

/var/folders/sv/qy8vlp51007g2k72ybwrhhqw0000gp/T/ipykernel_21930/2183993706.py:30: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df = df[~df[df.columns[0]].astype(str).str.contains(string, na=False)]
/var/folders/sv/qy8vlp51007g2k72ybwrhhqw0000gp/T/ipykernel_21930/2183993706.py:30: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df = df[~df[df.columns[0]].astype(str).str.contains(string, na=False)]
/var/folders/sv/qy8vlp51007g2k72ybwrhhqw0000gp/T/ipykernel_21930/2183993706.py:30: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df = df[~df[df.columns[0]].astype(str).str.contains(string, na=False)]
/var/folders/sv/qy8vlp51007g2k72ybwrhhqw0000gp/T/ipykernel_21930/2183993706.py:30: UserWarning: This pattern is interpreted as a regu

Finished processing file: csv/Ведомость остатков на 30.06.2022г. (сч. 21).csv

Processing file: csv/Ведомость остатков на 31.12.2022г. (сч. 21).csv
Removed 11 rows with empty first column.
Removed 1 rows containing 'Ведомость остатков ОС, НМА, НПА' in the first column.
Removed 1 rows containing 'Счет' in the first column.
Removed 1 rows containing 'КФО' in the first column.
Removed 1 rows containing 'КПС' in the first column.
Removed 1 rows containing 'ЦМО.Сотрудник' in the first column.
Removed 1 rows containing 'ЦМО.Место хранения' in the first column.
Removed 1 rows containing '№ п/п' in the first column.
Removed 3 rows containing 'Кузнецов Максим Сергеевич' in the first column.
Removed 0 rows containing '04100000000000000' in the first column.
Removed 0 rows containing 'Номенклатура' in the first column.
Removed 0 rows containing 'Кузнецов М. С. -' in the first column.
Removed 0 rows containing 'Единица измерения: рубль (код по ОКЕИ 383)' in the first column.
Removed 0 rows contain

/var/folders/sv/qy8vlp51007g2k72ybwrhhqw0000gp/T/ipykernel_21930/2183993706.py:30: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df = df[~df[df.columns[0]].astype(str).str.contains(string, na=False)]
/var/folders/sv/qy8vlp51007g2k72ybwrhhqw0000gp/T/ipykernel_21930/2183993706.py:30: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df = df[~df[df.columns[0]].astype(str).str.contains(string, na=False)]
/var/folders/sv/qy8vlp51007g2k72ybwrhhqw0000gp/T/ipykernel_21930/2183993706.py:30: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df = df[~df[df.columns[0]].astype(str).str.contains(string, na=False)]
/var/folders/sv/qy8vlp51007g2k72ybwrhhqw0000gp/T/ipykernel_21930/2183993706.py:30: UserWarning: This pattern is interpreted as a regu

Removed 1 rows starting with 'Оборотно-сальдовая ведомость по счету'.
Removed 824 rows with specific patterns in first column and empty second column.
Finished processing file: csv/Оборотно-сальдовая ведомость по сч. 101 за 1 кв. 2022г..csv

Processing file: csv/Оборотно-сальдовая ведомость по сч. 101 за 3 кв. 2022г..csv
Removed 12 rows with empty first column.
Removed 0 rows containing 'Ведомость остатков ОС, НМА, НПА' in the first column.
Removed 1 rows containing 'Счет' in the first column.
Removed 0 rows containing 'КФО' in the first column.
Removed 0 rows containing 'КПС' in the first column.
Removed 0 rows containing 'ЦМО.Сотрудник' in the first column.
Removed 0 rows containing 'ЦМО.Место хранения' in the first column.
Removed 0 rows containing '№ п/п' in the first column.
Removed 0 rows containing 'Кузнецов Максим Сергеевич' in the first column.
Removed 0 rows containing '04100000000000000' in the first column.
Removed 0 rows containing 'Номенклатура' in the first column.
Remov

/var/folders/sv/qy8vlp51007g2k72ybwrhhqw0000gp/T/ipykernel_21930/2183993706.py:30: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df = df[~df[df.columns[0]].astype(str).str.contains(string, na=False)]
/var/folders/sv/qy8vlp51007g2k72ybwrhhqw0000gp/T/ipykernel_21930/2183993706.py:30: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df = df[~df[df.columns[0]].astype(str).str.contains(string, na=False)]
/var/folders/sv/qy8vlp51007g2k72ybwrhhqw0000gp/T/ipykernel_21930/2183993706.py:30: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df = df[~df[df.columns[0]].astype(str).str.contains(string, na=False)]
/var/folders/sv/qy8vlp51007g2k72ybwrhhqw0000gp/T/ipykernel_21930/2183993706.py:30: UserWarning: This pattern is interpreted as a regu

Finished processing file: csv/Оборотно-сальдовая ведомость по сч. 101 за 2 кв. 2022г..csv

Processing file: csv/Оборотно-сальдовая ведомость по сч. 21 за 4 кв. 2022г..csv
Removed 12 rows with empty first column.
Removed 0 rows containing 'Ведомость остатков ОС, НМА, НПА' in the first column.
Removed 1 rows containing 'Счет' in the first column.
Removed 0 rows containing 'КФО' in the first column.
Removed 0 rows containing 'КПС' in the first column.
Removed 0 rows containing 'ЦМО.Сотрудник' in the first column.
Removed 0 rows containing 'ЦМО.Место хранения' in the first column.
Removed 0 rows containing '№ п/п' in the first column.
Removed 0 rows containing 'Кузнецов Максим Сергеевич' in the first column.
Removed 0 rows containing '04100000000000000' in the first column.
Removed 0 rows containing 'Номенклатура' in the first column.
Removed 2079 rows containing 'Кузнецов М. С. -' in the first column.
Removed 0 rows containing 'Единица измерения: рубль (код по ОКЕИ 383)' in the first colu

/var/folders/sv/qy8vlp51007g2k72ybwrhhqw0000gp/T/ipykernel_21930/2183993706.py:30: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df = df[~df[df.columns[0]].astype(str).str.contains(string, na=False)]
/var/folders/sv/qy8vlp51007g2k72ybwrhhqw0000gp/T/ipykernel_21930/2183993706.py:30: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df = df[~df[df.columns[0]].astype(str).str.contains(string, na=False)]


getting tables for union

In [147]:
import os
import pandas as pd
from collections import defaultdict

# Define the folder containing the CSV files
csv_folder = 'csv'

def get_table_structure(file_path):
    try:
        df = pd.read_csv(file_path)
        table_name = os.path.basename(file_path)
        field_names = tuple(sorted(df.columns))  # Sorting the columns to ensure consistent comparison
        
        return table_name, field_names
    except Exception as e:
        print(f"An error occurred while processing {file_path}: {e}")
        return None, None

# Dictionary to hold the structures and corresponding table names
structure_dict = defaultdict(list)

# Loop through all files in the csv folder
for filename in os.listdir(csv_folder):
    if filename.endswith('.csv'):
        file_path = os.path.join(csv_folder, filename)
        table_name, field_names = get_table_structure(file_path)
        
        if field_names:
            structure_dict[field_names].append(table_name)

# Prepare the result array
united_tables = {f"unitedTable{idx+1}": tables for idx, tables in enumerate(structure_dict.values())}

# Print the result
print(united_tables)


{'unitedTable1': ['Ведомость остатков на 31.03.2022г. (сч. 21).csv', 'Ведомость остатков на 30.09.2022г. (сч. 21).csv', 'Ведомость остатков на 30.06.2022г. (сч. 21).csv', 'Ведомость остатков на 31.12.2022г. (сч. 21).csv'], 'unitedTable2': ['Ведомость остатков на 30.06.2022г. (сч. 101).csv', 'Ведомость остатков на 31.03.2022г.(сч. 101).csv', 'Ведомость остатков на 30.09.2022г. (сч. 101).csv', 'Ведомость остатков на 31.12.2022г. (сч. 101).csv'], 'unitedTable3': ['Оборотная ведомость по сч. 105 за 1 кв. 2022г..csv', 'Оборотная ведомость по сч. 105 за 4 кв. 2022г..csv', 'Оборотная ведомость по сч. 105 за 3 кв. 2022г..csv', 'Оборотная ведомость по сч. 105 за 2 кв. 2022г..csv'], 'unitedTable4': ['Ведомость остатков на 31.03.2022 (сч. 105).csv', 'Ведомость остатков на 30.09.2022 (сч. 105).csv', 'Ведомость остатков на 31.12.2022 (сч. 105).csv', 'Ведомость остатков на 30.06.2022 (сч. 105).csv'], 'unitedTable5': ['Оборотно-сальдовая ведомость по сч. 21 за 1 кв. 2022г..csv', 'Оборотно-сальдовая в

In [148]:
import os
import pandas as pd

# Define the folder containing the CSV files
csv_folder = 'csv'

def print_table_structure(file_path):
    print(f"Processing file: {file_path}")
    try:
        df = pd.read_csv(file_path)
        table_name = os.path.basename(file_path)
        field_names = list(df.columns)
        
        print(f"Table name: {table_name}")
        print(f"Field names: {field_names}\n")
        
    except Exception as e:
        print(f"An error occurred while processing {file_path}: {e}")

# Loop through all files in the csv folder
for filename in os.listdir(csv_folder):
    if filename.endswith('.csv'):
        file_path = os.path.join(csv_folder, filename)
        print_table_structure(file_path)

print("Table structure printout complete!")


Processing file: csv/Ведомость остатков на 31.03.2022г. (сч. 21).csv
Table name: Ведомость остатков на 31.03.2022г. (сч. 21).csv
Field names: ['balance_statement_os_nma_npa', 'fixed_asset', 'signature', 'rasshifrovka_podpisi', 'inventory_number', 'okof', 'depreciation_group', 'depreciation_method', 'accounting_date', 'condition', 'useful_life', 'monthly_wear_norm', 'wear', 'balance_value', 'quantity', 'depreciation_amount', 'residual_value', 'account', 'doc_date', 'doc_account']

Processing file: csv/Ведомость остатков на 30.09.2022г. (сч. 21).csv
Table name: Ведомость остатков на 30.09.2022г. (сч. 21).csv
Field names: ['balance_statement_os_nma_npa', 'fixed_asset', 'signature', 'rasshifrovka_podpisi', 'inventory_number', 'okof', 'depreciation_group', 'depreciation_method', 'accounting_date', 'condition', 'useful_life', 'monthly_wear_norm', 'wear', 'balance_value', 'quantity', 'depreciation_amount', 'residual_value', 'account', 'doc_date', 'doc_account']

Processing file: csv/Ведомость

Romoveing some fields

In [149]:
import os
import pandas as pd

# Define the folder containing the CSV files
csv_folder = 'csv'

# Fields to be removed
fields_to_remove = ['sign','signature', 'podpis', 'dolzhnost', 'position', 'rasshifrovka_podpisi']

def remove_fields_from_csv(file_path, fields):
    """
    Removes specified fields from the CSV file.
    :param file_path: Path to the CSV file.
    :param fields: List of fields to be removed.
    """
    # Read the CSV file into a DataFrame
    df = pd.read_csv(file_path)
    
    # Drop the specified fields if they exist in the DataFrame
    df.drop(columns=[field for field in fields if field in df.columns], inplace=True)
    
    # Save the updated DataFrame back to the CSV file
    df.to_csv(file_path, index=False)
    print(f"Processed file: {file_path}")

# Iterate over all files in the csv_folder
for filename in os.listdir(csv_folder):
    if filename.endswith('.csv'):
        file_path = os.path.join(csv_folder, filename)
        remove_fields_from_csv(file_path, fields_to_remove)

print("Field removal complete!")


Processed file: csv/Ведомость остатков на 31.03.2022г. (сч. 21).csv
Processed file: csv/Ведомость остатков на 30.09.2022г. (сч. 21).csv
Processed file: csv/Ведомость остатков на 30.06.2022г. (сч. 101).csv
Processed file: csv/Оборотная ведомость по сч. 105 за 1 кв. 2022г..csv
Processed file: csv/Ведомость остатков на 31.03.2022г.(сч. 101).csv
Processed file: csv/Ведомость остатков на 31.03.2022 (сч. 105).csv
Processed file: csv/Ведомость остатков на 30.09.2022 (сч. 105).csv
Processed file: csv/Ведомость остатков на 30.09.2022г. (сч. 101).csv
Processed file: csv/Ведомость остатков на 30.06.2022г. (сч. 21).csv
Processed file: csv/Ведомость остатков на 31.12.2022г. (сч. 21).csv
Processed file: csv/Оборотная ведомость по сч. 105 за 4 кв. 2022г..csv
Processed file: csv/Ведомость остатков на 31.12.2022г. (сч. 101).csv
Processed file: csv/Оборотная ведомость по сч. 105 за 3 кв. 2022г..csv
Processed file: csv/Оборотная ведомость по сч. 105 за 2 кв. 2022г..csv
Processed file: csv/Оборотно-сальдо

In [150]:
import os
import pandas as pd
from collections import defaultdict

# Define the folder containing the CSV files
csv_folder = 'csv'

def get_table_structure(file_path):
    try:
        df = pd.read_csv(file_path)
        table_name = os.path.basename(file_path)
        field_names = tuple(df.columns)
        return table_name, field_names
    except Exception as e:
        print(f"An error occurred while processing {file_path}: {e}")
        return None, None

# Dictionary to hold field sets and corresponding table names
field_set_to_tables = defaultdict(list)

# Loop through all files in the csv folder
for filename in os.listdir(csv_folder):
    if filename.endswith('.csv'):
        file_path = os.path.join(csv_folder, filename)
        table_name, field_names = get_table_structure(file_path)
        if field_names:
            field_set_to_tables[field_names].append(table_name)

# Print the result
united_tables = [{field_set: table_names} for field_set, table_names in field_set_to_tables.items()]

# Display the result
for item in united_tables:
    for field_set, tables in item.items():
        print(f"Field set: {field_set}")
        print(f"Tables: {tables}\n")

print("Table structure analysis and grouping complete!")


Field set: ('balance_statement_os_nma_npa', 'fixed_asset', 'inventory_number', 'okof', 'depreciation_group', 'depreciation_method', 'accounting_date', 'condition', 'useful_life', 'monthly_wear_norm', 'wear', 'balance_value', 'quantity', 'depreciation_amount', 'residual_value', 'account', 'doc_date', 'doc_account')
Tables: ['Ведомость остатков на 31.03.2022г. (сч. 21).csv', 'Ведомость остатков на 30.09.2022г. (сч. 21).csv', 'Ведомость остатков на 30.06.2022г. (сч. 101).csv', 'Ведомость остатков на 31.03.2022г.(сч. 101).csv', 'Ведомость остатков на 30.09.2022г. (сч. 101).csv', 'Ведомость остатков на 30.06.2022г. (сч. 21).csv', 'Ведомость остатков на 31.12.2022г. (сч. 21).csv', 'Ведомость остатков на 31.12.2022г. (сч. 101).csv']

Field set: ('pp', 'inventory_item_number', 'reference_code', 'non_financial_asset_name', 'unit_of_measurement', 'quarter_balance_number_begin', 'quarter_balance_amount_begin', 'quarter_turnover_debit_number', 'quarter_turnover_debit_amount', 'quarter_turnover_cre

Upload data to db tables

In [2]:
import os
import re
import pandas as pd
import psycopg2
from collections import defaultdict
from unidecode import unidecode
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Define the folder containing the CSV files
csv_folder = 'csv'

# Database connection details from environment variables
db_name = os.getenv('DB_NAME', 'moshack')
db_user = os.getenv('DB_USER', 'postgres')
db_password = os.getenv('DB_PASSWORD', 'postgres')
db_host = os.getenv('DB_HOST', 'localhost')
db_port = os.getenv('DB_PORT', '5432')

def get_table_structure(file_path):
    try:
        df = pd.read_csv(file_path)
        table_name = os.path.basename(file_path)
        field_names = tuple(df.columns)
        return table_name, field_names
    except Exception as e:
        print(f"An error occurred while processing {file_path}: {e}")
        return None, None

def longest_common_substring(strs):
    if not strs:
        return ''
    substr = ''
    for i in range(len(strs[0])):
        for j in range(i + 1, len(strs[0]) + 1):
            candidate = strs[0][i:j]
            if all(candidate in s for s in strs) and len(candidate) > len(substr):
                substr = candidate
    return substr

def create_table_name(file_names, index):
    common_substr = longest_common_substring(file_names)
    transliterated = unidecode(common_substr).replace(' ', '_')
    # Remove any characters that are not allowed in SQL identifiers
    table_name = re.sub(r'[^a-zA-Z0-9_]', '', transliterated)
    return f"{table_name}_{index}".lower()

def create_table_query(table_name, columns):
    columns_with_types = ', '.join([f'"{col}" TEXT' for col in columns])
    return f'CREATE TABLE IF NOT EXISTS {table_name} ({columns_with_types});'

def upload_data_to_table(conn, table_name, file_path):
    df = pd.read_csv(file_path)
    df.columns = [col.lower() for col in df.columns]  # Ensure all column names are lowercase
    with conn.cursor() as cur:
        for row in df.itertuples(index=False, name=None):
            values_placeholder = ', '.join(['%s'] * len(row))
            insert_query = f'INSERT INTO {table_name} VALUES ({values_placeholder})'
            cur.execute(insert_query, row)
    conn.commit()

def main():
    field_set_to_tables = defaultdict(list)
    for filename in os.listdir(csv_folder):
        if filename.endswith('.csv'):
            file_path = os.path.join(csv_folder, filename)
            table_name, field_names = get_table_structure(file_path)
            if field_names:
                field_set_to_tables[field_names].append(filename)

    united_tables = [{field_set: table_names} for field_set, table_names in field_set_to_tables.items()]

    conn = psycopg2.connect(
        dbname=db_name,
        user=db_user,
        password=db_password,
        host=db_host,
        port=db_port
    )

    for index, item in enumerate(united_tables):
        for field_set, tables in item.items():
            table_name = create_table_name(tables, index)
            create_query = create_table_query(table_name, field_set)
            with conn.cursor() as cur:
                cur.execute(create_query)
                cur.execute(f'TRUNCATE TABLE {table_name};')  # Clean the table only once
            for table_file in tables:
                file_path = os.path.join(csv_folder, table_file)
                upload_data_to_table(conn, table_name, file_path)

    conn.close()

if __name__ == '__main__':
    main()

print("Data upload to PostgreSQL complete!")


Data upload to PostgreSQL complete!


In [11]:
import psycopg2
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Database connection details from environment variables
db_name = os.getenv('DB_NAME', 'moshack')
db_user = os.getenv('DB_USER', 'postgres')
db_password = os.getenv('DB_PASSWORD', 'postgres')
db_host = os.getenv('DB_HOST', 'localhost')
db_port = os.getenv('DB_PORT', '5432')

# Connect to the PostgreSQL database
conn = psycopg2.connect(
    dbname=db_name,
    user=db_user,
    password=db_password,
    host=db_host,
    port=db_port
)

# Create a cursor object
cur = conn.cursor()

# Execute the query to get the list of tables
cur.execute("""
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public'
ORDER BY table_name;
""")

# Fetch all results
tables = cur.fetchall()

# Print the list of tables
for table in tables:
    print(table[0])

# Close the cursor and connection
cur.close()
conn.close()


oborotnaia_vedomost_po_sch_105_za__1
oborotnosaldovaia_vedomost_po_sch__3
vedomost_ostatkov_na_3_0
vedomost_ostatkov_na_3_2


uploading dictionary

In [13]:
import pandas as pd
import os

# Define source and destination paths
src_folder = 'src2'
dest_folder = 'csv2'
excel_file = 'КПГЗ ,СПГЗ, СТЕ.xlsx'
csv_file = 'dictionary.csv'

# Create the destination folder if it does not exist
if not os.path.exists(dest_folder):
    os.makedirs(dest_folder)

# Construct full file paths
excel_path = os.path.join(src_folder, excel_file)
csv_path = os.path.join(dest_folder, csv_file)

# Read the Excel file
try:
    df = pd.read_excel(excel_path)
    
    # Convert the DataFrame to a CSV file
    df.to_csv(csv_path, index=False, encoding='utf-8')
    print(f"Successfully converted {excel_file} to {csv_file}")
except Exception as e:
    print(f"An error occurred: {e}")


Successfully converted КПГЗ ,СПГЗ, СТЕ.xlsx to dictionary.csv


uploading to db

In [15]:
import pandas as pd
import os
import re
import psycopg2
from transliterate import translit
from sqlalchemy import create_engine, text

# Database configuration
db_name = os.getenv('DB_NAME', 'moshack')
db_user = os.getenv('DB_USER', 'postgres')
db_password = os.getenv('DB_PASSWORD', 'postgres')
db_host = os.getenv('DB_HOST', 'localhost')
db_port = os.getenv('DB_PORT', '5432')

# Define source and destination paths
csv_folder = 'csv2'
csv_file = 'dictionary.csv'
csv_path = os.path.join(csv_folder, csv_file)

# Function to sanitize and transliterate column names
def sanitize_column_names(columns):
    sanitized_columns = []
    for col in columns:
        col = translit(col, 'ru', reversed=True)  # Transliterate to ASCII
        col = col.strip().lower()  # Lowercase and strip spaces
        col = re.sub(r'[^\w\s]', '', col)  # Remove non-alphanumeric characters
        col = col.replace(' ', '_')  # Replace spaces with underscores
        sanitized_columns.append(col)
    return sanitized_columns

# Read the CSV file
df = pd.read_csv(csv_path)

# Rename the columns
df.columns = sanitize_column_names(df.columns)

# Create database engine
engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')

# Function to create table if it does not exist
def create_table_if_not_exists(table_name, df, engine):
    columns = df.columns
    column_types = ', '.join([f'"{col}" TEXT' for col in columns])
    create_table_query = f'CREATE TABLE IF NOT EXISTS {table_name} ({column_types});'
    
    with engine.connect() as connection:
        connection.execute(text(create_table_query))
        connection.commit()

# Define the table name
table_name = 'dictionary'

# Create the table if it does not exist
create_table_if_not_exists(table_name, df, engine)

# Upload the data to the table
df.to_sql(table_name, engine, if_exists='replace', index=False)

print("Data upload to PostgreSQL complete!")


Data upload to PostgreSQL complete!
